<a href="https://colab.research.google.com/github/olinml2024/notebooks/blob/main/ML24_Day13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Motivating Word Embeddings

## Toy Problem

We've done a mini-lecture to explain some key ideas behind the notion of word embeddings.  Now we're going to actually see some in action.  In this notebook, we are going to start with a toy problem to help us wrap our minds around the key ideas.

We'll begin by embedding not words but single letters.  We decided to start with this because we wanted to make our first example as simple as possible.  First, we will create a dataset consisting of 10,000 letter patterns.  Each letter pattern will either consist of two vowels, a consonant, and then two more vowels (VVCVV) or two consontants, a vowel, and then two more consontants (CCVCC).

After generating these patterns, we'll print some out so you can see what they look like.

In [ ]:
import torch
import random
import numpy as np

# patterns are VVCVV CCVCC
n_train = 10000
vowels = set('aeiou')
consonants = set('bcdfghjklmnpqrstvwxyz')

def make_patterns(n):
    X = []
    for i in range(n):
        if random.randint(0, 1) == 1:
            random_vowel = random.choice(tuple(vowels))
            four_random_consonants = np.random.choice(tuple(consonants), 4)
            X.append([four_random_consonants[0], four_random_consonants[1], random_vowel, four_random_consonants[2], four_random_consonants[3]])
        else:
            random_consonant = random.choice(tuple(consonants))
            four_random_vowels = np.random.choice(tuple(vowels), 4)
            X.append([four_random_vowels[0], four_random_vowels[1], random_consonant, four_random_vowels[2], four_random_vowels[3]])
    return X

X = make_patterns(n_train)
print(X)

Next, were going to train a character embedding.  We begin by defining two dictionaries.  The first, called ``tokenize`` is for turning each of the 26 letters in to a token (represented as an integer from 0-25).  The second, called ``detokenize``, implements the inverse operation (from integer to letter).

In [ ]:
tokenize = dict(zip(list(vowels) + list(consonants), list(range(26))))
detokenize = dict(zip(tokenize.values(), tokenize.keys()))

print(tokenize['a'])                    # should be 0
print(detokenize[tokenize['a']])        # should get back to 'a'

Now, we can define what's called the skip-gram model.  The skip-gram model, in this case, attempts to use the center letter to predict the surrounding letters.  For example, if we have a CCVCC pattern, we would want to be able to take the center letter (vowel) as input and predict the identity of the surrounding consonants.

You may be thinking this is an impossible task.  You'd be right.  Let's take a minute to discuss this.

Nevertheless, we can define a model that does its best to solve the task.  Let's take a look at this model in detail.

In [ ]:
from torch.nn import Module
from torch import nn
import matplotlib.pyplot as plt

class SkipGram(Module):
    def __init__(self, n_embd=2):
        """
        Create the SkipGram model.
        Args:
          n_embd: the number of dimensions to use for our character embeddings.
        """
        super().__init__()
        # hardcoding 26 (note: could also use len(tokenize))
        self.embedding = nn.Embedding(26, n_embd)
        self.output = nn.Linear(n_embd, 26)

    def __call__(self, x):
        """
        The forward pass of our model
        Args:
          x: the center letter (represented as an integer from 0-25)

        Returns: the log probability of each of the 26 letters.
        """
        x = self.embedding(x)
        x = self.output(x)
        x = nn.functional.log_softmax(x, dim=1)
        return x

model = SkipGram()

# start off by tokenize each letter in each of our 10,000 patterns
X_tok = torch.tensor([[tokenize[letter] for letter in x] for x in X])
# we are predicting the first two and last two characters
y_tok = torch.hstack((X_tok[:,0:2], X_tok[:,3:5]))
X_tok = X_tok[:,2]

optimizer = torch.optim.Adam(model.parameters(), lr=10**-2)
n_epochs = 500
losses = np.zeros((n_epochs,1))
for n in range(n_epochs):
    log_probs = model(X_tok)
    # this is fancy syntax grabs the corresponding log probabilities corresponding
    # to the letters that actually surrounded our center letter.
    loss = -log_probs.gather(1,y_tok).mean()
    model.zero_grad()
    loss.backward()
    losses[n] = loss.item()
    optimizer.step()

plt.plot(range(n_epochs), losses)
plt.ylabel('negative log likelihood')
plt.xlabel('epoch')
plt.show()

Next, we can look at the embeddings themselves as a way to understand what the model learned.

In [ ]:
import matplotlib.pyplot as plt
embedded = model.embedding(torch.tensor(list(tokenize.values()))).detach().numpy()
fig, ax = plt.subplots()
ax.scatter(embedded[:,0], embedded[:,1])
text = list(tokenize.keys())
for i in range(embedded.shape[0]):
    ax.annotate(text[i], (embedded[i,0], embedded[i,1]))

## Skip-grams on Words

We can extend these same ideas to a little bit more interesting problem.  We begin with an old dataset of text called [the Penn Tree Bank](https://catalog.ldc.upenn.edu/LDC99T42).

You can use the link above for a more complete description, but a very high-level overview is that the dataset contains "one million words of 1989 Wall Street Journal material."  This dataset is convenient in that the sample we will load has already been tokenized (case removed, uncommon words replaced by a special symbol, etc.).

We'll repeat largely the same steps as we did for our letter example but we'll use word as a our unit of analysis. We'll also build in support for using the GPU, batches of data, and allow the context window size to be adjusted.


In [ ]:
!wget https://raw.githubusercontent.com/tomsercu/lstm/refs/heads/master/data/ptb.train.txt -O ptb.train.txt
!wget https://raw.githubusercontent.com/tomsercu/lstm/refs/heads/master/data/ptb.test.txt -O ptb.test.txt
with open('ptb.train.txt') as f:
    text = f.read()
words = text.split()
unique_words = set(words)

In [ ]:
tokenize_words = dict(zip(list(unique_words), list(range(len(unique_words)))))
detokenize_words = dict(zip(tokenize_words.values(), tokenize_words.keys()))

class SkipGram_words(Module):
    def __init__(self, n_embd=2):
        super().__init__()
        self.embedding = nn.Embedding(len(unique_words), n_embd)
        self.linear_1 = nn.Linear(n_embd, len(unique_words))

    def __call__(self, x):
        x = self.embedding(x)
        x = self.linear_1(x)
        x = nn.functional.log_softmax(x, dim=1)
        return x

model = SkipGram_words(n_embd=100)
# use a bigger window size (this seemed to give interesting results)
sliding_window_size = 9
assert(sliding_window_size % 2 == 1)
center_idx = (sliding_window_size-1)//2

all_contexts = []
# use sliding window
for i in range(0, len(words)-sliding_window_size+1):
    all_contexts.append([tokenize_words[w] for w in words[i:i+sliding_window_size]])

X_tok = torch.tensor(all_contexts)
y_tok = torch.hstack((X_tok[:,:center_idx], X_tok[:,center_idx+1:]))
X_tok = X_tok[:,center_idx]

optimizer = torch.optim.Adam(model.parameters(), lr=10**-3, weight_decay=10**-4)

trainset = torch.utils.data.TensorDataset(X_tok, y_tok)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=2048, shuffle=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
n_epochs = 10
for n in range(n_epochs):
    running_loss = 0.0
    total_processed = 0
    for i, data in enumerate(trainloader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        log_probs = model(inputs.to(device))
        loss = -log_probs.gather(1,labels.to(device)).mean()
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        total_processed += inputs.shape[0]
        if i % 200 == 0:
            print(total_processed/len(all_contexts))
            print(f"{n} {running_loss/(i+1)}")

We can use the embeddings of the 500 most common words and look for the most similar word for each.

In [ ]:
from sklearn.neighbors import NearestNeighbors
from collections import Counter

c = Counter(words)
most_common_words, most_common_word_inds = \
    zip(*map(lambda k: (k[0], tokenize_words[k[0]]),
         c.most_common(500)))
most_common_word_inds_as_tensor = torch.tensor(most_common_word_inds).to(device)
common_word_embeddings = model.embedding(most_common_word_inds_as_tensor).detach().cpu().numpy()
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(common_word_embeddings)
distances, indices = nbrs.kneighbors(common_word_embeddings)
closest_map = dict(zip(most_common_words, [most_common_words[ind] for ind in indices[:,1]]))

In [ ]:
closest_map